In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [2]:
list=[[0,'Town Street','01-02-2017 06:01:00'],
      [0,'Town Street','01-02-2017 06:03:00'],
      [0,'Town Street','01-02-2017 06:18:00'],
      [0,'Town Street','01-02-2017 06:06:00'],
      [0,'Town Street','02-02-2017 10:01:00'],
      [0,'Town Street','02-02-2017 10:05:00']]

df=spark.createDataFrame(list,['col1','col2','timestamp'])

df.show()

+----+-----------+-------------------+
col1| col2| timestamp|
+----+-----------+-------------------+
 0|Town Street|01-02-2017 06:01:00|
 0|Town Street|01-02-2017 06:03:00|
 0|Town Street|01-02-2017 06:05:00|
 0|Town Street|01-02-2017 06:06:00|
 0|Town Street|02-02-2017 10:01:00|
 0|Town Street|02-02-2017 10:05:00|
+----+-----------+-------------------+

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("date").orderBy(F.col("timestamp").cast("long")).rangeBetween(Window.currentRow,60*5)

df.withColumn("timestamp", F.to_timestamp("timestamp",'MM-dd-yyyy HH:mm:ss'))\
  .withColumn("date", F.to_date("timestamp"))\
  .withColumn('collect', F.size(F.collect_list("timestamp").over(w))).filter("collect>1")\
  .select(F.date_format("date","yyyy-MM-dd").alias("date"), F.array(F.date_format("timestamp","HH:mm:ss"),F.col("collect")).alias("time"))\
  .orderBy("date").show()

+----------+-------------+
 date| time|
+----------+-------------+
2017-01-02|[06:01:00, 4]|
2017-01-02|[06:05:00, 2]|
2017-01-02|[06:03:00, 3]|
2017-02-02|[10:01:00, 2]|
+----------+-------------+

In [4]:
list=[[[1,2,3,4]]]

df=spark.createDataFrame(list,['list'])

df.show(truncate=False)

+------------+
list |
+------------+
[1, 2, 3, 4]|
+------------+

In [5]:
df.withColumn("list2", F.col("list").getItem(0))\
  .withColumn("list3", F.col("list")[0])\
  .withColumn("list4", F.element_at("list", 1)).show()

+------------+-----+-----+-----+
 list|list2|list3|list4|
+------------+-----+-----+-----+
[1, 2, 3, 4]| 1| 1| 1|
+------------+-----+-----+-----+

In [6]:
DEF= ['LB','LWB','RB','LCB','RCB','CB','RWB']
FWD= ['RF','LF','LW','RS','RW','LS','CF','ST'] 
MID= ['LCM','LM','RDM','CAM','RAM','RCM','CM','CDM','RM','LAM','LDM'] 

list=[['RF'],
     ['RDM'],
     ['CB']]

df=spark.createDataFrame(list,['Position'])

df.show()

+--------+
Position|
+--------+
 RF|
 RDM|
 CB|
+--------+

In [7]:
from pyspark.sql import functions as F
df.withColumn("Position_2", F.when(F.col("Position").rlike('RF$'),F.lit('DEF'))\
                             .when(F.col("Position").rlike('RDM$'),F.lit('FWD'))\
                             .otherwise(F.lit('MID'))).show()

+--------+----------+
Position|Position_2|
+--------+----------+
 RF| DEF|
 RDM| FWD|
 CB| MID|
+--------+----------+

In [8]:
from pyspark.sql import functions as F
from pyspark.sql.functions import when

DEF= ['LB','LWB','RB','LCB','RCB','CB','RWB']
FWD= ['RF','LF','LW','RS','RW','LS','CF','ST'] 
MID= ['LCM','LM','RDM','CAM','RAM','RCM','CM','CDM','RM','LAM','LDM'] 

df.withColumn("DEF", F.array(*[F.lit(x) for x in DEF]))\
  .withColumn("FWD", F.array(*[F.lit(x) for x in FWD]))\
  .withColumn("Position_2", F.when(F.expr("""array_contains(DEF,Position)""")==True, F.lit('DEF'))\
                             .when(F.expr("""array_contains(FWD,Position)""")==True, F.lit('FWD'))\
                              .otherwise(F.lit("MID"))).drop("DEF","MID","FWD").show()

+--------+----------+
Position|Position_2|
+--------+----------+
 RF| FWD|
 RDM| MID|
 CB| DEF|
+--------+----------+

In [9]:
df.withColumn("position_2", F.when(F.arrays_overlap(F.array(*(F.lit(x) for x in DEF)),F.array("Position")),F.lit('DEF'))\
                             .when(F.arrays_overlap(F.array(*(F.lit(x) for x in FWD)),F.array("Position")),F.lit('FWD'))\
                             .otherwise(F.lit('MID'))).show()

+--------+----------+
Position|position_2|
+--------+----------+
 RF| FWD|
 RDM| MID|
 CB| DEF|
+--------+----------+

In [10]:
list=[['b','7/1/2019',12,1],           
      ['a','6/9/2019',0,5],           
      ['c','1/1/2019',0,0]]  

df=spark.createDataFrame(list,['USER','DATE','COUNT_COL1','COUNT_COL2'])

df.show()

+----+--------+----------+----------+
USER| DATE|COUNT_COL1|COUNT_COL2|
+----+--------+----------+----------+
 b|7/1/2019| 12| 1|
 a|6/9/2019| 0| 5|
 c|1/1/2019| 0| 0|
+----+--------+----------+----------+

In [12]:
df.withColumn('cnt', F.sum((df[c[0]]>0).astype("int") for c in df.dtypes if c[1] in ['int','long'])).show()

--------------------------------------------------------------------------- 
 Py4JError Traceback (most recent call last)
 <command-2954689015007897> in <module> 
 ----> 1 df . withColumn ( 'cnt' , F . sum ( ( df [ c [ 0 ] ] > 0 ) . astype ( "int" ) for c in df . dtypes if c [ 1 ] in [ 'int' , 'long' ] ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in _ (col) 
 42 def _ ( col ) : 
 43 sc = SparkContext . _active_spark_context
 ---> 44 jc = getattr ( sc . _jvm . functions , name ) ( col . _jc if isinstance ( col , Column ) else col ) 
 45 return Column ( jc ) 
 46 _ . __name__ = name

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 330 raise Py4JError(
 331 "An error occurred while calling {0}{1}{2}. Trace:\n{3}\n" . 
 --> 332 format(target_id, ".", name, value))
 333 else : 
 334 raise Py4JError(

 Py4JError : An error occurred while calling z:org.apache.spark.sql.functions.sum. Trace:
py4j.Py4JException: Method sum([class java.util.ArrayList]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:341)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:362)
	at py4j.Gateway.invoke(Gateway.java:289)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [13]:
for i in df.dtypes:
  print(i[0],i[1])

USER string
DATE string
COUNT_COL1 bigint
COUNT_COL2 bigint

In [14]:

df.withColumn("arr", F.array(*[F.col(i[0]) for i in df.dtypes if i[1] in ['int','bigint']]))\
  .withColumn("DESIRED COLUMN", F.expr("""size(filter(arr,x->x!=0))""")).drop("arr").show()

+----+--------+----------+----------+--------------+
USER| DATE|COUNT_COL1|COUNT_COL2|DESIRED COLUMN|
+----+--------+----------+----------+--------------+
 b|7/1/2019| 12| 1| 2|
 a|6/9/2019| 0| 5| 1|
 c|1/1/2019| 0| 0| 0|
+----+--------+----------+----------+--------------+

In [15]:
df.withColumn("arr", F.array(*[F.col(i[0]) for i in df.dtypes if i[1] in ['int','bigint']]))\
  .withColumn("DESIRED COLUMN", F.expr("""size(filter(arr,x->x!=0))""")).drop("arr").show()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3747450522555419> in <module> 
 1 
 ----> 2 df . withColumn ( "arr" , F . sum ( F . col ( i [ 0 ] ) for i in df . dtypes if i [ 1 ] in [ 'int' , 'bigint' ] ) ) \ 
 3 . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in _ (col) 
 42 def _ ( col ) : 
 43 sc = SparkContext . _active_spark_context
 ---> 44 jc = getattr ( sc . _jvm . functions , name ) ( col . _jc if isinstance ( col , Column ) else col ) 
 45 return Column ( jc ) 
 46 _ . __name__ = name

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 499 java_list = ArrayList ( ) 
 500 for element in object : 
 --> 501 java_list . add ( element ) 
 502 return java_list
 503 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 498 ArrayList = JavaClass ( "java.util.ArrayList" , gateway_client ) 
 499 java_list = ArrayList ( ) 
 --> 500 for element in object : 
 501 java_list . add ( element ) 
 502 return java_list

 /databricks/spark/python/pyspark/sql/column.py in __iter__ (self) 
 342 
 343 def __iter__ ( self ) : 
 --> 344 raise TypeError ( "Column is not iterable" ) 
 345 
 346 # string methods 

 TypeError : Column is not iterable

In [16]:
df.select(F.isnan("COUNT_COL1")).show()

+-----------------+
isnan(COUNT_COL1)|
+-----------------+
 false|
 false|
 false|
+-----------------+

In [17]:
df.withColumn('cnt', F.sum((df[c[0]]>0).astype("int") for c in df.dtypes if c[1] in ['int','long'])).show()

--------------------------------------------------------------------------- 
 Py4JError Traceback (most recent call last)
 <command-2954689015007896> in <module> 
 ----> 1 df . withColumn ( 'cnt' , F . sum ( ( df [ c [ 0 ] ] > 0 ) . astype ( "int" ) for c in df . dtypes if c [ 1 ] in [ 'int' , 'long' ] ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in _ (col) 
 42 def _ ( col ) : 
 43 sc = SparkContext . _active_spark_context
 ---> 44 jc = getattr ( sc . _jvm . functions , name ) ( col . _jc if isinstance ( col , Column ) else col ) 
 45 return Column ( jc ) 
 46 _ . __name__ = name

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 330 raise Py4JError(
 331 "An error occurred while calling {0}{1}{2}. Trace:\n{3}\n" . 
 --> 332 format(target_id, ".", name, value))
 333 else : 
 334 raise Py4JError(

 Py4JError : An error occurred while calling z:org.apache.spark.sql.functions.sum. Trace:
py4j.Py4JException: Method sum([class java.util.ArrayList]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:341)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:362)
	at py4j.Gateway.invoke(Gateway.java:289)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [18]:
df=spark.createDataFrame([(1,5,"This"),(2,10,"is"),(3,12,"a"),(1,7,"string"),(2,4,"oreo")],["usr","sec","scrpt"])
df.show()

+---+---+------+
usr|sec| scrpt|
+---+---+------+
 1| 5| This|
 2| 10| is|
 3| 12| a|
 1| 7|string|
 2| 4| oreo|
+---+---+------+

In [19]:
df.groupBy("usr").agg(F.sort_array(F.collect_list(F.struct("sec","scrpt"))).alias("concated")).show()

+---+--------------------+
usr| concated|
+---+--------------------+
 1|[[5, This], [7, s...|
 3| [[12, a]]|
 2|[[4, oreo], [10, ...|
+---+--------------------+

In [20]:
df.groupBy("usr").agg(F.array_join(F.expr("""transform(sort_array(collect_list(struct(sec,scrpt)),True), x -> x.scrpt)""")," ").alias("concated")).orderBy("usr").show(10,False)

+---+-----------+
usr|concated |
+---+-----------+
1 |This string|
2 |oreo is |
3 |a |
+---+-----------+

In [21]:
list=[[ 1, 0],
        [1,  1],
        [1,  1],
        [1,  0],
        [1,  1],
        [1,  1],
        [2,  1],
        [2,  1],
        [3,  0],
        [3,  1],
        [3,  1],
        [3,  1]]

df=spark.createDataFrame(list,['A','B'])

df.show()

+---+---+
 A| B|
+---+---+
 1| 0|
 1| 1|
 1| 1|
 1| 0|
 1| 1|
 1| 1|
 2| 1|
 2| 1|
 3| 0|
 3| 1|
 3| 1|
 3| 1|
+---+---+

In [22]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("A").orderBy("A")
w2=Window().partitionBy("C").orderBy("A")
df.withColumn("C", F.sum(F.when(F.col("B")==F.lit(0), F.lit(1)).otherwise(F.lit(0))).over(w))\
  .withColumn("C", F.row_number().over(w2)).show()

+---+---+---+
 A| B| C|
+---+---+---+
 2| 1| 1|
 2| 1| 2|
 3| 0| 1|
 3| 1| 2|
 3| 1| 3|
 3| 1| 4|
 1| 0| 1|
 1| 1| 2|
 1| 1| 3|
 1| 0| 4|
 1| 1| 5|
 1| 1| 6|
+---+---+---+

In [23]:

    +---+---+---+
    |  A|  B|  C|
    +---+---+---+
    |  1|  0|  1|
    |  1|  1|  2|
    |  1|  1|  3|
    |  1|  0|  4|
    |  1|  1|  5|
    |  1|  1|  6|
    |  3|  0|  1|
    |  3|  1|  2|
    |  3|  1|  3|
    |  3|  1|  4|
    |  2|  1|  1|
    |  2|  1|  2|
    +---+---+---+

In [24]:
list= [v1     |2012-01-10| [5,2] |
      [v1     |2012-01-11| [2,5] |
      [v1     |2012-01-12| [3,2] |
      [v2     |2012-01-12| [2]   |
       v2     |2012-01-11| [1,2] |
| v2     |2012-01-13| [1]   |
+--------+----------+-------+

In [25]:
list=[[[5,2]],
      [[2,5]],
     [[2]]]

df=spark.createDataFrame(list,['list'])

df.show()

+------+
 list|
+------+
[5, 2]|
[2, 5]|
 [2]|
+------+

In [26]:
df.withColumn("diff", F.when(F.size('list')==2, F.expr("""transform(array(list),x-> x[0]-x[1])""")[0])\
                       .otherwise(F.lit(0))).show()

+------+----+
 list|diff|
+------+----+
[5, 2]| 3|
[2, 5]| -3|
 [2]| 0|
+------+----+

In [27]:
list=[[[['componenţa', 'parlamentului:', 'a', 'se', 'vedea', 'procesul-verbal'],['membership', 'of', 'parliament:', 'see', 'minutes']]]]

df=spark.createDataFrame(list,['list'])

df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------+
list |
+----------------------------------------------------------------------------------------------------------+
[[componenţa, parlamentului:, a, se, vedea, procesul-verbal], [membership, of, parliament:, see, minutes]]|
+----------------------------------------------------------------------------------------------------------+

In [28]:
df.withColumn("list", F.expr("""transform(list,x-> filter(x, y-> y!='vedea'))""")).show(truncate=False)

+---------------------------------------------------------------------------------------------------+
list |
+---------------------------------------------------------------------------------------------------+
[[componenţa, parlamentului:, a, se, procesul-verbal], [membership, of, parliament:, see, minutes]]|
+---------------------------------------------------------------------------------------------------+

In [29]:
df.withColumn("list",\
F.expr("""filter(list,x-> array_contains(x,'vedea')!=True)""")).show(truncate=False)

+---------------------------------------------+
list |
+---------------------------------------------+
[[membership, of, parliament:, see, minutes]]|
+---------------------------------------------+

In [30]:
df.withColumn("list", F.expr("""filter(list,x-> IF(array_contains(x,'vedea'),array_remove(list,x),array(x)))""")).show(truncate=False)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1072.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'filter(`list`, lambdafunction((IF(array_contains(namedlambdavariable(), 'vedea'), array_remove(`list`, namedlambdavariable()), array(namedlambdavariable()))), namedlambdavariable()))' due to data type mismatch: argument 2 requires boolean type, however, 'lambdafunction((IF(array_contains(namedlambdavariable(), 'vedea'), array_remove(`list`, namedlambdavariable()), array(namedlambdavariable()))), namedlambdavariable())' is of array<array<string>> type.;;
Project [filter(list#380, lambdafunction(if (array_contains(lambda x#400, vedea)) array_remove(list#380, lambda x#400) else array(lambda x#400), lambda x#400, false)) AS list#399]
+- LogicalRDD [list#380], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:1